In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from analysis.analyzers.data_segmenter import DataSegmenter
from analysis.transformations.ohlcv_to_ohlcv_and_deltas import ohlcv_to_ohlcv_and_deltas
from analysis.segmenters.overlapping_windows_segmenter import striding_segmenter

segmenter = striding_segmenter(length=7, stride=3)

m = DataSegmenter(
    ticker='SPY',
    transformation=ohlcv_to_ohlcv_and_deltas,
    segmenter=segmenter
)

m.load_historical_for_ticker('SPY')
m.transform_data()
m.save_transformed_data()
m.segment_data()

In [4]:
m.save_segmented_data()

In [5]:
from pathlib import Path

segmenter_name = 'test'
segmented_data_path = Path.cwd()/'studies'/'SPY'/m.transformation_name/'segmented'/segmenter_name/'data.pkl'
m.load_segmented_data(segmented_data_path)

In [6]:
print(len(m.segments))

68378


In [11]:
from analysis.categorizers.gaussian_mixtures import GaussianMixtureCategorizer

vectors = [seg.to_numpy().flatten() for seg in m.segments]

categorizer = GaussianMixtureCategorizer(data=vectors)

In [24]:
categorizer.categorize_large_dataset()

Optimal clusters: 2
Sample silhouette: 0.359
Full data silhouette: 0.358


array([0, 0, 0, ..., 1, 1, 1], shape=(68378,), dtype=int32)

In [19]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

sil_score = silhouette_score(vectors, categorizer.labels)

# Calinski-Harabasz Index (higher = better)
ch_score = calinski_harabasz_score(vectors, categorizer.labels)

# Davies-Bouldin Index (lower = better)
db_score = davies_bouldin_score(vectors, categorizer.labels)

print(f"Silhouette Score: {sil_score:.3f}")
print(f"Calinski-Harabasz Index: {ch_score:.2f}")
print(f"Davies-Bouldin Index: {db_score:.3f}")



Silhouette Score: -0.187
Calinski-Harabasz Index: 128.73
Davies-Bouldin Index: 0.956
